In [1]:
%classpath add jar ../../konduit.jar

## Model Link
The dl4j model is taken from the dl4j-example here: https://github.com/eclipse/deeplearning4j-examples/blob/master/mvn-project-template/src/main/java/org/deeplearning4j/examples/sample/LeNetMNIST.java

In [2]:
%%bash
konduit serve -id dl4j-mnist -c dl4j.json -rwm -b

Starting konduit server...
Using classpath: /root/konduit/bin/../konduit.jar
INFO: Running command /root/miniconda/jre/bin/java -Dkonduit.logs.file.path=/root/.konduit-serving/command_logs/dl4j-mnist.log -Dlogback.configurationFile=/tmp/logback-run_command_a6000ad26ed94583.xml -jar /root/konduit/bin/../konduit.jar run --instances 1 -s inference -c dl4j.json -Dserving.id=dl4j-mnist
For server status, execute: 'konduit list'
For logs, execute: 'konduit logs dl4j-mnist'



In [3]:
%%bash
konduit logs dl4j-mnist -l 100

15:00:54.356 [vert.x-worker-thread-0] ERROR o.n.common.config.ND4JClassLoading - Cannot find class [org.nd4j.linalg.jblas.JblasBackend] of provided class-loader.
15:00:54.357 [vert.x-worker-thread-0] ERROR o.n.common.config.ND4JClassLoading - Cannot find class [org.canova.api.io.data.DoubleWritable] of provided class-loader.
15:00:54.407 [vert.x-worker-thread-0] ERROR o.n.common.config.ND4JClassLoading - Cannot find class [org.nd4j.linalg.jblas.JblasBackend] of provided class-loader.
15:00:54.407 [vert.x-worker-thread-0] ERROR o.n.common.config.ND4JClassLoading - Cannot find class [org.canova.api.io.data.DoubleWritable] of provided class-loader.
15:00:54.601 [vert.x-worker-thread-0] INFO  org.nd4j.nativeblas.NativeOpsHolder - Number of threads used for linear algebra: 8
15:00:54.602 [vert.x-worker-thread-0] WARN  o.n.l.c.nativecpu.CpuNDArrayFactory - *********************************** CPU Feature Check Warning ***********************************
15:00:54.602 [vert.x-worker-thread-0] W

In [4]:
%%bash
konduit list


Listing konduit servers...

 #   | ID                             | TYPE       | URL                  | PID     | STATUS     
 1   | dl4j-mnist                     | inference  | localhost:39487      | 1517    | started    




In [5]:
%%html
<img src="test-image.jpg"/>

In [6]:
%%bash
konduit predict dl4j-mnist -it multipart "image=@test-image.jpg"

{
  "layer5" : [ [ 1.845163E-5, 1.8346094E-6, 0.31436875, 0.43937472, 2.6101702E-8, 0.24587035, 5.9430695E-6, 3.3270408E-4, 6.3698195E-8, 2.708706E-5 ] ],
  "prob" : 0.439374715089798,
  "index" : 3,
  "label" : "3"
}



In [7]:
%%bash
konduit stop dl4j-mnist

Stopping konduit server 'dl4j-mnist'
Application 'dl4j-mnist' terminated with status 0

